In [4]:
import os
os.chdir( "/home/lucasfuzato/CODING/WINE" )

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from src.aux_fun import *

tests_df = pd.DataFrame( columns = [ "max_depth" , "algo" ] )

depths = [ 5 , 10 , 15 , 20 , 25 , 30 ]
algos  = [ "gini" , "entropy" ]
for i , ( max_depth , algo ) in enumerate( product( depths , algos ) ):
    tests_df.loc[ i ] = pd.Series( [ max_depth , algo ] , [ "max_depth" , "algo" ] )
print( tests_df ) 

   max_depth     algo
0          5     gini
1          5  entropy
2         10     gini
3         10  entropy
4         15     gini
5         15  entropy
6         20     gini
7         20  entropy
8         25     gini
9         25  entropy
10        30     gini
11        30  entropy


In [6]:
from sklearn.tree import DecisionTreeClassifier as dec_tree
seed_val = 47

X , y , X_2d , columns = class_get_data()

scores = [ "f1_score" , "precision" , "recall" ]
metrics = [ "mean" , "std" ]
data = { tup:list() for tup in product( scores , metrics ) }

for simu_id , ( max_depth , algo ) in enumerate( product( depths , algos ) ):

    lst = []
    fold_iter = get_prepared_data( X , y , seed = seed_val , smote = True )
    for ( X_train , y_train ) , ( X_test , y_test ) in fold_iter:

        tree = dec_tree( criterion = algo , max_depth = max_depth )
        tree.fit( X_train , y_train )

        ts_cls = tree.predict( X_test )
        ts_f1 , prec , recall = bin_f1_score( ts_cls , y_test )

        lst.append( ( ts_f1 , prec , recall ) )
    
    arr = np.array( lst )
    means = arr.mean( axis = 0 )
    data[ ( "f1_score", "mean" ) ].append( means[ 0 ] )
    data[ ( "precision", "mean" ) ].append( means[ 1 ] )
    data[ ( "recall", "mean" ) ].append( means[ 2 ] )

    stds  = arr.std( axis = 0 )
    data[ ( "f1_score", "std" ) ].append( stds[ 0 ] )
    data[ ( "precision", "std" ) ].append( stds[ 1 ] )
    data[ ( "recall", "std" ) ].append( stds[ 2 ] )

print( pd.DataFrame( data ) )
print()

    f1_score           precision              recall          
        mean       std      mean       std      mean       std
0   0.506401  0.026361  0.395296  0.016015  0.710316  0.083692
1   0.508614  0.029872  0.397445  0.018611  0.709486  0.068009
2   0.561114  0.036747  0.455384  0.036901  0.733009  0.044153
3   0.560240  0.022728  0.464787  0.027342  0.706355  0.020862
4   0.578874  0.028693  0.477424  0.026319  0.736873  0.046136
5   0.588103  0.029132  0.491752  0.022898  0.732985  0.051032
6   0.575828  0.024754  0.482157  0.023570  0.715810  0.036852
7   0.577454  0.029877  0.481342  0.029302  0.722841  0.039828
8   0.574739  0.028326  0.482662  0.028677  0.711866  0.040748
9   0.595699  0.031778  0.502328  0.026107  0.733003  0.050886
10  0.594300  0.024497  0.502374  0.028209  0.729085  0.032032
11  0.587219  0.032743  0.488159  0.025587  0.736928  0.046266

